In [1]:
# import nest_asyncio
# nest_asyncio.apply()
import os
import django
import inspect

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
    
from books.models import Book
from books import utils

from asgiref.sync import sync_to_async


In [2]:
b = Book.objects.raw('SELECT * FROM books_book WHERE title="Jacob Jones"')
from django.core.serializers import serialize
serialize('json', b)

'[{"model": "books.book", "pk": 45, "fields": {"title": "Jacob Jones", "publication_date": "2001-04-05", "price": 62.0, "serial_number": "73b96721-df46-4516-9bee-b5629a4ca8d1", "description": "Short price tell huge popular indeed. One certain every beyond both.\\nOption mother send good. Space professor grow hair site agent word. Race anyone inside plant however.\\nLikely rise if likely drive law often. Show read goal add.\\nBehind main certain growth. Build lose top nature president."}}]'

In [11]:
from django.contrib.admin.options import modelform_factory

f = modelform_factory(Book, fields=['serial_number'])
Book._meta.verbose_name
f['serial_number']

TypeError: 'ModelFormMetaclass' object is not subscriptable

In [95]:
from faker import Faker
from time import time
import time as T
import random
import string

import hashlib


def generate_random_string(length):
    letters = string.ascii_letters
    random_string = ''.join(random.sample(string.ascii_letters,length))
    return random_string

f = Faker()
t1 = time()
for i in range(1000000):
    # ''.join(random.sample(string.ascii_letters,10))
    # f.uuid4()
    data = f"Hello, World!{T.time()}".encode()
    x.update(data)
    x.hexdigest()
    f.text(10)
    
t2 = time()
t2-t1

7.442434072494507

In [29]:
generate_random_string(12)

'dxlbGLpcnEJM'

In [26]:
''.join(random.sample(string.ascii_letters,10))

'KGZDIEQgyi'

In [102]:
import time
'1232.11251'.format(.2)
round(time.time(),2)

1695200912.47